In [90]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import LSTM,Dense,Input,ConvLSTM2D,Reshape,Activation,Lambda,Softmax,Concatenate
from keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout,RepeatVector,Reshape,Activation,Lambda,Softmax
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
import re
import pandas as pd
from keras import backend as K
import keras.backend as kerback
#from keras.models import Model
from tensorflow.keras.models import Model
from keras.preprocessing.text import one_hot
import tensorflow 

In [91]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy import random
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense

In [92]:
batch_size=60

In [93]:
def prepare_data(f):
    data = pd.read_csv(f, sep="\n",header=None)
    list_listword=[]#liste des listes des mots de chaque tweet
    list_tweets=data[0].values.tolist()#liste des tweets
    l=[]
    for text in list_tweets:
        text = re.sub(r'http\S+', '', text)   # Remove URLs
        text = re.sub(r'—', ' ', text) 
        text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
        text = text.strip(" ")   # Remove whitespace resulting from above
        text = re.sub(r' +', ' ', text)   # Remove redundant spaces
        l.append(text)
    
    return l

In [94]:
list_tweets=prepare_data('C:/Users/LENOVO/Desktop/PCD/tweet.txt')
print(len(list_tweets[3]))

254


In [95]:
def tokenizer_data(list_tweets):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list_tweets)
    sequences = tokenizer.texts_to_sequences(list_tweets)
    vocab_size=len(tokenizer.word_index)
    return sequences, tokenizer

def max_tweet(sequences):
    max_length=0
    for i in range(len(sequences)):
        if(len(sequences[i])>max_length):
            max_length=len(sequences[i])
        #print("max",max_length)
        #print("seq=",sequences)
        #print(vocab_size)
    return max_length

sequences, tokenizer=tokenizer_data(list_tweets)
print(sequences[0])
print(len(sequences[0])) #nombre d'element de tweet zero
nb_tweet=len(sequences)
#print(len(tokenizer.word_index))
max_length=max_tweet(sequences)#nombre d'element de tout les tweet pour les mettres egaux
print(max_length) #############

[146, 7, 46, 5, 2, 147, 148, 106, 3, 17, 149, 107, 1, 348, 349, 30, 350, 58, 351, 352, 3, 353, 354, 208, 52, 355, 356, 357, 7, 358, 3, 209, 47, 18, 359, 7, 360, 22, 4, 27]
40
57


In [140]:
# define the standalone generator model
def build_generator(n_outputs):#max_length nombre d'element das une tweet
    model = Sequential()
    x=Input(shape=(n_outputs,1))
    z0=Input(shape=(n_outputs,))
    z=Embedding(57,1)(z0)
    x0=Concatenate(axis=2)([x,z])
    x1=LSTM(15, activation='relu',input_dim=n_outputs)(x0)
    #15 nombre de neurone ,batch_size=n_batch=20 
    # repeat vector
    x2=RepeatVector(n_outputs)(x1)
    # decoder layer
    x3=LSTM(15, activation='relu', return_sequences=True)(x2)
    x4=TimeDistributed(Dense(1))(x3)
    unstacked = Lambda(lambda x: tensorflow.unstack(x, axis=2))(x4)
    dense_outputs = [Dense(n_outputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)

    #generated_tweet=model(x0)
    model = Model([x,z0],merged )
    #model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    print('generator')
    print(model.summary())
    return  model

In [141]:
# define the standalone discriminator model
def build_discriminator(n_inputs):
    model = Sequential()
    x=Input(shape=(n_inputs,1)) #n_imput nombre des element 
    #x1=Flatten()(x0)
    z0=Input(shape=(n_inputs,))
    z=Embedding(57,1)(z0)
    x0=Concatenate(axis=2)([x,z])
    x2=LSTM(15, activation='relu', input_dim=n_inputs,batch_size=30)(x0)
    x3=RepeatVector(n_inputs)(x2)
    x4=LSTM(15, activation='relu', return_sequences=True)(x3)
    x5=TimeDistributed(Dense(1))(x4)
    unstacked = Lambda(lambda x: tensorflow.unstack(x, axis=2))(x5)
    dense_outputs = [Dense(n_inputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)
    x1=Flatten()(merged)
    x6=Dense(1, activation='sigmoid')(x1)
    model = Model([x,z0], x6)
    print(model.summary())
    #model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])
    return model

In [142]:
# define the combined generator and discriminator model
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    print(model.summary())
    return model

In [143]:
def generate_real_samples():
    list_tweets=prepare_data('C:/Users/LENOVO/Desktop/PCD/tweet.txt')
    sequences, tokenizer=tokenizer_data(list_tweets)
    #print(sequences[0])
    #print("len=",len(sequences[0]))
    max_length=max_tweet(sequences)
    #print("max_length",max_length)
    #print(sequences[0].shape)
    t =sequences[:batch_size]#on a prix les 30 1er tweets
    t = pad_sequences(t, maxlen=max_length, padding='post')
    #print("len",len(t[0]))
    #print(t[0])
    #print(t[0].shape)
   
    #print("len1",len(sequences))
    #print("len2",len(t))
    #X=X.reshape(30, 70, 1) #33 array chaque array contient 70 array chaque array contient 1 element
    #print(t[0]) #262 elements
    #print(t)
    return t

In [144]:
# Create batch labels to use when calling train_on_batch
y_real = ones((batch_size,1))
y_fake = zeros((batch_size,1))

In [145]:
# Build and compile the discriminator
discriminator = build_discriminator(max_length)
#discriminator.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])

# Build and compile the combined model
generator = build_generator(max_length)
gan_model = define_gan(generator, discriminator)

Model: "functional_71"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_62 (InputLayer)           [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_61 (InputLayer)           [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 57, 1)        57          input_62[0][0]                   
__________________________________________________________________________________________________
concatenate_20 (Concatenate)    (None, 57, 2)        0           input_61[0][0]                   
                                                                 embedding_20[0][0]   

AssertionError: Could not compute output Tensor("lambda_75/stack:0", shape=(None, 57, 1), dtype=float32)

In [102]:
def sample_tweet(epoch,noise,g_model):
    result = g_model.predict(noise)
    #print(result)
    #print(result.shape)
    #print(len(result))
    res=[]
    predicted_word =""
    for j in range(57):
        #predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == int(result[0][j]):
                predicted_word = predicted_word+' '+word
                break
    res.append(predicted_word)
    print(res)
    
    #for i in res:
    #    print(i)
    return res

In [103]:
def disc_text(model_disc, seed_text):
    #print(seed_text)
    sequences, tokenizer=tokenizer_data(seed_text)
    sequences= pad_sequences(sequences, maxlen=max_length, padding='post')
    y = model_disc.predict(sequences)
    #print(y)
    #for i in y:
    if(i<0.7):
        print("fake")
    else:
        print("real")

In [104]:
# Store the losses
d_losses = []
g_losses = []
sample_period = 100 # every `sample_period` steps generate and save some data"
Epoch=1000
#x_real= generate_real_samples()
######################## Main training loop###########################
for i in range(Epoch):
        ########################
        ###Train Discriminator##
        #######################
        # prepare real samples
        x_real= generate_real_samples()

        # Generate fake tweets
        noise=random.randn(batch_size,1)
        x_fake= generator.predict(noise)
        #print("x_fake shape",x_fake.shape)
        
        # Train the discriminator
        # both loss and accuracy are returned
        d_loss_real, d_acc_real=discriminator.train_on_batch(x_real, y_real)
        d_loss_fake, d_acc_fake =discriminator.train_on_batch(x_fake, y_fake)
        d_loss = 0.5 * (d_loss_real + d_loss_fake)
        d_acc  = 0.5 * (d_acc_real + d_acc_fake)
        
        #######################
        ### Train generator ###
        #######################
    
        y_gan = ones((batch_size, 1))
        noise=random.randn(batch_size,57)
        g_loss=gan_model.train_on_batch(noise, y_gan)
        
        # Save the losses
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
        # evaluate discriminator on real examples
        #acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
        # evaluate discriminator on fake examples
        #acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
        
        #if i % 2 == 100:#after 100 epoch we save
        #if i % sample_period == 0:
        print(f"epoch: {i+1}, d_loss: {d_loss:.2f}, \
        d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
        noiseg=random.randn(1,57)
        sample_tweet(i,noiseg, generator)
        #disc_text(discriminator, tex)

AssertionError: in user code:

    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\LE